learn some pydantic first

In [5]:
import pydantic
from pydantic import BaseModel, Field
pydantic.__version__

'2.5.3'

In [11]:
class User(BaseModel):
    id: int
    name: str

test = {
    "id": 1,
    "name": "John Doe"

}

test_fake = {
    "id": "1",
    "name": "John Doe",
    "height": 1.8   

}

user = User(**test)
user_fake = User(**test_fake)

print(user), print(user_fake), isinstance(user_fake.id, int)


id=1 name='John Doe'
id=1 name='John Doe'


(None, None, True)

In [13]:
from pydantic import conlist

class Address(BaseModel):
    street: str
    city: str
    state: str
    zip: str

class Employee(BaseModel):
    id: int
    name: str
    address: Address
    email: str

class Owner(BaseModel):
    id: int
    name: str
    address: Address
    email: str

class Restuarant(BaseModel):
    id: int
    name: str
    owner: Owner
    employees: conlist(Employee, min_length=2)


In [15]:
Restuarant.model_json_schema()

{'$defs': {'Address': {'properties': {'street': {'title': 'Street',
     'type': 'string'},
    'city': {'title': 'City', 'type': 'string'},
    'state': {'title': 'State', 'type': 'string'},
    'zip': {'title': 'Zip', 'type': 'string'}},
   'required': ['street', 'city', 'state', 'zip'],
   'title': 'Address',
   'type': 'object'},
  'Employee': {'properties': {'id': {'title': 'Id', 'type': 'integer'},
    'name': {'title': 'Name', 'type': 'string'},
    'address': {'$ref': '#/$defs/Address'},
    'email': {'title': 'Email', 'type': 'string'}},
   'required': ['id', 'name', 'address', 'email'],
   'title': 'Employee',
   'type': 'object'},
  'Owner': {'properties': {'id': {'title': 'Id', 'type': 'integer'},
    'name': {'title': 'Name', 'type': 'string'},
    'address': {'$ref': '#/$defs/Address'},
    'email': {'title': 'Email', 'type': 'string'}},
   'required': ['id', 'name', 'address', 'email'],
   'title': 'Owner',
   'type': 'object'}},
 'properties': {'id': {'title': 'Id', 'ty

In [16]:
res = Restuarant(
    id=1,
    name="The Palace",
    owner=Owner(
        id=1,
        name="John Doe",
        address=Address(
            street="1234 Elm St",
            city="Springfield",
            state="IL",
            zip="62701"
        ),
        email="email.com",
    ),
    employees=[
        Employee(
            id=1,
            name="Jane Doe",
            address=Address(
                street="1234 Elm St",
                city="Springfield",
                state="IL",
                zip="62701"
            ),
            email="email.com"
        ),
        Employee(
            id=2,
            name="Jim Doe",
            address=Address(
                street="1234 Elm St",
                city="Springfield",
                state="IL",
                zip="62701"
            ),
            email="email.com"
        )
    ]




)

In [20]:
for emp in res.employees:
    print(emp.name, emp.address.city, emp.email)
    


Jane Doe Springfield email.com
Jim Doe Springfield email.com


In [25]:
from pydantic import EmailStr, field_validator

#field validators.. validates JUST a single field

class Owner(BaseModel):
    name: str
    email: EmailStr

    @field_validator("name")
    @classmethod
    def name_must_contain_space(cls, v):
        if " " not in v:
            raise ValueError("Must contain a space you idiot")
        return v.title()
    


try:
    owner_instance = Owner(name="john doe", email="johndoe@gmail.com")
    print(owner_instance)
except Exception as e:
    print(e)


name='John Doe' email='johndoe@gmail.com'


In [36]:
class Person(BaseModel):
    first_name: str
    last_name: str
    age: int
wes = Person(first_name="Wes", last_name="Stone", age=40)
wes

Person(first_name='Wes', last_name='Stone', age=40)

In [42]:
from pydantic import ValidationError

try:
    Person(first_name="Wes", last_name="Stone", age="40")
except ValidationError as e:
    print(e)


wes = Person(first_name="notWes", last_name="Stone", age=40)

wes.first_name = "Wes"
wes.first_name

'Wes'

In [45]:
try:
    Person(first_name="Wes", last_name=100, age="junk")
except ValidationError as e:
    exceptions = e

exceptions.errors(), exceptions.json()


([{'type': 'string_type',
   'loc': ('last_name',),
   'msg': 'Input should be a valid string',
   'input': 100,
   'url': 'https://errors.pydantic.dev/2.5/v/string_type'},
  {'type': 'int_parsing',
   'loc': ('age',),
   'msg': 'Input should be a valid integer, unable to parse string as an integer',
   'input': 'junk',
   'url': 'https://errors.pydantic.dev/2.5/v/int_parsing'}],
 '[{"type":"string_type","loc":["last_name"],"msg":"Input should be a valid string","input":100,"url":"https://errors.pydantic.dev/2.5/v/string_type"},{"type":"int_parsing","loc":["age"],"msg":"Input should be a valid integer, unable to parse string as an integer","input":"junk","url":"https://errors.pydantic.dev/2.5/v/int_parsing"}]')

In [47]:
data = {
    "first_name": "Wes",
    "last_name": "Stone",
    "age": 40
}

p = Person(**data)
p

Person(first_name='Wes', last_name='Stone', age=40)

In [54]:
data = '''{
    "first_name": "Wes",
    "last_name": "Stone",
    "age": 40
}
'''

p = Person.model_validate_json(data)

p

Person(first_name='Wes', last_name='Stone', age=40)

In [56]:
try:
    p = Person(age=40)
except ValidationError as e:
    print(e)


2 validation errors for Person
first_name
  Field required [type=missing, input_value={'age': 40}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.5/v/missing
last_name
  Field required [type=missing, input_value={'age': 40}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.5/v/missing


In [62]:
class Person(BaseModel):
    first_name: str = "Wes"
    last_name: str = "Stone"
    age: int = 20


Person.model_fields, Person()


({'first_name': FieldInfo(annotation=str, required=False, default='Wes'),
  'last_name': FieldInfo(annotation=str, required=False, default='Stone'),
  'age': FieldInfo(annotation=int, required=False, default=20)},
 Person(first_name='Wes', last_name='Stone', age=20))

In [66]:
class Person(BaseModel):
    first_name: str | None = None
    last_name: str
    age: int

p = Person(last_name="Stone", age=40)
p

Person(first_name=None, last_name='Stone', age=40)